# Ratio
A class that implements fractional numbers. The `Ratio` class provides exact arithmetic for representing exact musical quantities such as proportional (metric) time, duration, and 'just' tuning intervals. Ratios can be created from two integers or from a string. Ratios are compared and combined using the standard math operators.

In [15]:
class Ratio:
    pass

## Ratio.\_\_init\_\_
#### Creates a Ratio from integers, a floating point number, or a string name.
 * `Ratio(int, int)` - creates a ratio from an integer numerator and denominator.
 * `Ratio(int)` - creates a ratio from an integer numerator with the denominator set to 1.
 * `Ratio(float)` - creates a ratio from a floating point number (see: `as_integer_ratio()`)
 * `Ratio(string)` -  creates a ratio from a string 'num/den'. Both num and den must produce valid integers.

```
@param num If only num is specified it must be either an integer, float, or a string containing a valid ratio expression 'a/b'. If both num and den are provided they must both be integer value.
@param den If specified den must be a non-zero integer denominator
```

Upon construction the new ratio will always be expressed in its most simple form, for example `Ratio(6,12)` will become `Ratio(1/2)`, See: `gcd()`. If both the numerator and denominator are negative the ratio should be converted to positive by the constructor.

The constructor should raise a `TypeError` if the num or den is not a integer, string or float and a `DivisionByZero` error if the denominator is 0.

In [11]:
    def __init__(self, num, den=None):
        pass